In [1]:
import numpy as np
import pandas as pd

In [2]:
k = 100
split_pair_path = "../data/split_test/new_split_pair_test_k_{0}.txt".format(k)
train_data_path = "../data/split_test/train_set.txt"
test_data_path = "../data/split_test/test_tweets_unlabeled_dataframe_v1_4.txt"


train_data = pd.read_csv(train_data_path, sep='\t', header=None)
train_label = np.array(train_data.iloc[:,0]).astype('int')
test_data = pd.read_csv(test_data_path)
split_data = pd.read_csv(split_pair_path, sep='\t')

In [3]:
# from probability file, get top k candidates
probability_file_path = "../data/split_test/test_results_k_100.csv"

probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]


In [4]:
%%time
top = k
new_top = top
probability_threshold = 1
new_candidate_list = [None] * int(len(split_data)/top)
counter = 0
for i in range(0, len(split_data), top):
    new_train_data_single_split = split_data[i:(i+top)]
    new_train_label_single_split = np.array(new_train_data_single_split.candidate_label)
    probability_single_split = probability_file[i:(i+top)]
    sorted_index = np.array(np.argsort(probability_single_split)[::-1][:new_top])
    sorted_probability_split = np.array(probability_single_split)[sorted_index]
    new_train_label_single_split = new_train_label_single_split[sorted_index]
    filted_probability_split = sorted_probability_split[sorted_probability_split > probability_threshold]
    candidate_label = np.array([new_train_label_single_split[0]])
    if 0 != len(filted_probability_split):
        candidate_label = new_train_label_single_split[:len(filted_probability_split)]
        #new_candidate_list.append(candidate_label)
    new_candidate_list[counter] = candidate_label
    counter += 1
    

CPU times: user 18 s, sys: 12.8 ms, total: 18 s
Wall time: 18 s


In [5]:
new_top_predict_label_list = np.zeros((len(new_candidate_list), 1)).astype('int')
for i in range(0, len(new_candidate_list)):
    predict_label = np.argmax(np.bincount(new_candidate_list[i]))
    new_top_predict_label_list[i] = predict_label

In [6]:
output_path = "../data/split_test/test_predict_label_k_{0}".format(k)
np.save(output_path, new_top_predict_label_list)

In [7]:
new_top_predict_label_list

array([[6353],
       [ 249],
       [6492],
       ...,
       [4181],
       [9205],
       [  82]])